In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("GEMINI_API_KEY")

In [3]:
from google import genai

client = genai.Client(api_key=api_key)

response = client.models.generate_content(
    model="gemini-2.0-flash-lite", contents="Hôm nay là ngày bao nhiêu?"
)
print(response.text)

Hôm nay là ngày 1 tháng 5 năm 2024.



# Full text fucking search lets go

In [17]:
import unicodedata, re

def normalize(text):
    text = text.lower()
    text = unicodedata.normalize('NFC', text)
    text = re.sub(r'[^a-z0-9ăâđêôơư\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [23]:
from pyvi import ViTokenizer

def load_stopwords(path="stopwords.txt"):
    with open(path, encoding="utf-8") as f:
        stops = {line.strip() for line in f
                 if line.strip() and not line.startswith("#")}
    return stops

stopwords = load_stopwords()

def remove_stopwords(sentence, stopwords=stopwords):
    tokenized = ViTokenizer.tokenize(sentence)
    tokens = tokenized.split()
    filtered = [t for t in tokens if t not in stopwords]
    return " ".join(filtered)

In [24]:
def preprocess_input(user_input):
    input = normalize(user_input)
    input = remove_stopwords(input)
    return input

In [98]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

def retrieval(query_str, index="vietnamnet", top_k=5):
    body = {
        "query": {
            "bool": {
                "should": [
                    {
                        "match_phrase": {
                            "content": {
                                "query": query_str,
                                "slop": 1,
                                "boost": 3
                            }
                        }
                    },
                    {
                        "multi_match": {
                            "query": query_str,
                            "fields": ["title^2", "summary^1.5", "content"],
                            "fuzziness": "AUTO"
                        }
                    }
                ]
            }
        }
    }
    res = es.search(index=index, body=body, size=top_k)
    hits = res.get("hits", {}).get("hits", [])
    return [hit["_source"] for hit in hits]

def answer(question, client):
    q = preprocess_input(question)
    docs = retrieval(q, top_k=5)
    context = "\n\n".join(f"{str(d)}" for d in docs)
    # print(context)
    # print("="*100)
    return client.models.generate_content(
        model="gemini-2.0-flash-lite",
        contents=(
            f"Trả lời câu hỏi sau: “{question}”\n"
            f"Dùng những gì bạn biết kết hợp với thông tin truy vấn được, các câu hỏi bạn không thấy trong phần thông tin liên quan hãy thông báo cho người dùng và cố gắng trả lời bằng dữ liệu bạn có"
            f"Dưới đây là các đoạn thông tin liên quan:\n{context}\n\n"
            "Hãy trả lời ngắn gọn, rõ ràng."
        )
    ).text


In [100]:
print(answer("Nội dung nghị định 52 là gì?", client))

C:\Users\Saphyiera\AppData\Local\Temp\ipykernel_14508\3378729784.py:30: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  res = es.search(index=index, body=body, size=top_k)


Nghị định 52/2024/NĐ-CP quy định về thanh toán không dùng tiền mặt. Các nội dung chính của nghị định bao gồm:

*   Quy định về tiền điện tử, bao gồm định nghĩa, hình thức (ví điện tử, thẻ trả trước) và đối tượng cung ứng.
*   Quy định về thanh toán quốc tế.
*   Sửa đổi, bổ sung các quy định về tài khoản thanh toán (mở, sử dụng, ủy quyền, phong tỏa, đóng tài khoản...).
*   Quy định về dịch vụ thanh toán không qua tài khoản thanh toán.



In [101]:
print(answer("Giá vàng ngày 15/3", client))

C:\Users\Saphyiera\AppData\Local\Temp\ipykernel_14508\3378729784.py:30: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  res = es.search(index=index, body=body, size=top_k)


Tôi xin lỗi, tôi không có thông tin về giá vàng ngày 15/3/2024. Tuy nhiên, tôi có thể cung cấp giá vàng ngày 15/3/2025 theo thông tin tôi tìm thấy:

*   **Giá vàng nhẫn:**
    *   SJC: 94,2 - 95,7 triệu đồng/lượng (mua - bán)
    *   Doji: 94,9 - 96,3 triệu đồng/lượng (mua - bán)
*   **Giá vàng miếng SJC:**
    *   SJC TP.HCM và Doji Hà Nội/TP.HCM: 94,3 - 95,8 triệu đồng/lượng (mua - bán)
*   **Giá vàng thế giới:** 2.984,5 USD/ounce (giao ngay)


In [105]:
print(answer("Giá vàng ngày 20/11", client))

C:\Users\Saphyiera\AppData\Local\Temp\ipykernel_14508\3378729784.py:30: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  res = es.search(index=index, body=body, size=top_k)


Tôi không có thông tin về giá vàng ngày 20/11/2024. Tuy nhiên, tôi có thể cung cấp cho bạn giá vàng ngày 20/11/2023:

*   **Giá vàng trong nước:**
    *   Giá vàng SJC:
        *   Mua vào: 70.150.000 - 70.050.000 đồng/lượng
        *   Bán ra: 70.870.000 - 70.750.000 đồng/lượng
    *   Giá vàng DOJI:
        *   Mua vào: 70.100.000 - 69.950.000 đồng/lượng
        *   Bán ra: 70.850.000 - 70.750.000 đồng/lượng
*   **Giá vàng quốc tế:** Khoảng 1.980,6 USD/ounce.

Xin lưu ý rằng giá vàng có thể thay đổi trong ngày. Để có thông tin chính xác nhất, bạn nên tham khảo các nguồn tin tức tài chính uy tín vào thời điểm hiện tại.



In [106]:
print(answer("Giá vàng ngày hôm nay", client))

C:\Users\Saphyiera\AppData\Local\Temp\ipykernel_14508\3378729784.py:30: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  res = es.search(index=index, body=body, size=top_k)


Tôi không có thông tin về "Giá vàng ngày hôm nay". Vui lòng cung cấp thêm thông tin để tôi có thể giúp bạn.



In [107]:
print(answer("Hôm nay là ngày bao nhiêu", client))

C:\Users\Saphyiera\AppData\Local\Temp\ipykernel_14508\3378729784.py:30: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  res = es.search(index=index, body=body, size=top_k)


Tôi không có thông tin về ngày hôm nay là ngày bao nhiêu. Tuy nhiên, tôi có thể cung cấp một số thông tin liên quan đến các ngày đã qua và sắp tới:

*   Ngày 10/9/2024: MB ký kết hợp tác với Công đoàn Y tế Việt Nam.
*   Ngày 11/09/2024: Bài báo được đăng tải.
*   Ngày 27/02/1955: Ngày Thầy thuốc Việt Nam (sẽ kỷ niệm 70 năm vào 27/02/2025).
*   Ngày 04/11/1994: Ngày thành lập MB (sẽ kỷ niệm 30 năm vào 04/11/2024).



In [83]:
def decompose(question, client):
    prompt = (
        "Bạn là trợ lý thông minh. "
        "Hãy tách câu sau thành các câu hỏi con rõ ràng, ngắn gọn, ít câu hỏi con nhất có thể:\n"
        f"“{question}”\n"
        "Trả về JSON list, ví dụ: [\"q1\",\"q2\"]"
    )
    resp = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt
    ).text
    print(resp)
    return resp

def answer_multi(question, client):
    subs = decompose(question, client)
    answers = []
    for sub in subs:
        a = answer(sub, client)
        answers.append({"question": sub, "answer": a})
    return answers


In [84]:
decompose("Giá vàng ngày 15/3 và 11/3", client)

```json
[
  "Giá vàng ngày 15/3 là bao nhiêu?",
  "Giá vàng ngày 11/3 là bao nhiêu?"
]
```



'```json\n[\n  "Giá vàng ngày 15/3 là bao nhiêu?",\n  "Giá vàng ngày 11/3 là bao nhiêu?"\n]\n```\n'

In [86]:
decompose("Giá vàng ngày hôm nay và hôm qua", client)

```json
[
  "Giá vàng hôm nay là bao nhiêu?",
  "Giá vàng hôm qua là bao nhiêu?"
]
```



'```json\n[\n  "Giá vàng hôm nay là bao nhiêu?",\n  "Giá vàng hôm qua là bao nhiêu?"\n]\n```\n'

In [ ]:
import re
import datetime
from dateparser.search import search_dates

BASE_DATE = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=7)))

RELATIVE_VN = {
    r'\bhôm nay\b': 0,
    r'\bngày mai\b': 1,
    r'\bngày kia\b': 2,
    r'\bhôm qua\b': -1,
    r'\bngày hôm qua\b': -1,
}

RE_DYNAMIC = [
    (re.compile(r'(\d+)\s+ngày\s+trước', flags=re.IGNORECASE), -1),
    (re.compile(r'(\d+)\s+ngày\s+sau',   flags=re.IGNORECASE), +1),
]

def normalize_dates(text, base_date=BASE_DATE):
    for pattern, offset in RELATIVE_VN.items():
        def _fix(m):
            dt = base_date + datetime.timedelta(days=offset)
            return dt.strftime("%d/%m/%Y")
        text = re.sub(pattern, _fix, text, flags=re.IGNORECASE)

    for regex, direction in RE_DYNAMIC:
        def _dyn(m):
            n = int(m.group(1))
            dt = base_date + datetime.timedelta(days=direction * n)
            return dt.strftime("%d/%m/%Y")
        text = regex.sub(_dyn, text)

    results = search_dates(
        text,
        languages=['vi'],
        settings={
            'PREFER_DATES_FROM': 'past',
            'RELATIVE_BASE': base_date,
            'RETURN_AS_TIMEZONE_AWARE': False
        }
    ) or []

    for expr, dt in sorted(results, key=lambda x: -len(x[0])):
        formatted = dt.strftime("%d/%m/%Y")
        esc = re.escape(expr)
        text = re.sub(rf"\b{esc}\b", formatted, text, flags=re.IGNORECASE)

    return text

print(normalize_dates("Hôm nay là hôm nay, 4 ngày trước là ngày mai"))

07/05/2025 là 07/05/2025, 03/05/2025 là 08/05/2025


# Later shit: broadcast model for document embeding to vector search